# 20181112
Room-1
Motion: 1: Draw-1; 2: Draw-2; 3: Draw-3; 4: Draw-4; 5: Draw-5; 6: Draw-6; 7: Draw-7; 8: Draw-8; 9: Draw-9; 0: Draw-0;
Tx-Rx Location(meter): Tx(0, 0) Rx_1(0.455, -0.455) Rx_2(1.365, -0.455) Rx_3(2.0, 0) Rx_4(-0.455, 0.455) Rx_5(-0.455, 1.365) Rx_6(0, 2.0)
Torso Location(meter): 1(1.365, 0.455) 2(0.455, 0.455) 3(0.455, 1.365) 4(1.365, 1.365) 5(0.91, 0.91)
​
# 20181116
Room-1
Motion: 1: Draw-1; 2: Draw-2; 3: Draw-3; 4: Draw-4; 5: Draw-5; 6: Draw-6; 7: Draw-7; 8: Draw-8; 9: Draw-9; 0: Draw-0;
Tx-Rx Location(meter): Tx(0, 0) Rx_1(0.455, -0.455) Rx_2(1.365, -0.455) Rx_3(2.0, 0) Rx_4(-0.455, 0.455) Rx_5(-0.455, 1.365) Rx_6(0, 2.0)
Torso Location(meter): 1(1.365, 0.455) 2(0.455, 0.455) 3(0.455, 1.365) 4(1.365, 1.365) 5(0.91, 0.91)

In [1]:
import os  
import shutil 
import random  
import re
from tqdm import tqdm
import threading
from extract_csi import get_scaled_csi, extract_csi
import scipy.signal as signal
import scipy.io as scio
import numpy as np

# 给文件添加label

In [16]:
# for f in os.listdir('/home/zut_csi/tomding/RCNN/data_gait/room1/1'):
#     # '/home/zut_csi/tomding/RCNN/data_qinghua/20181112/user2'):
#     if f.endswith('.dat'): # 文件
#         with open(os.path.join('/home/zut_csi/tomding/RCNN/data_processed', f.replace('dat', 'txt')), 'w') as w:
#             w.write(re.findall(re.compile(r'user.*-(.*?)-.*dat', re.S), f)[0])
#                                           #user2-(.*?)-.*-.*-*-r.*', re.S), f)[0])

path = '/home/zut_csi/tomding/RCNN/data_processed/all'
for f in os.listdir(path):
    # '/home/zut_csi/tomding/RCNN/data_qinghua/20181112/user2'):
    if f.endswith('.mat'): # 文件
        with open(os.path.join(path, f.replace('mat', 'txt')), 'w') as w:
            w.write(re.findall(re.compile(r'.*user.*-(.*?)-.*-r.*', re.S), f)[0])
                                          #user2-(.*?)-.*-.*-*-r.*', re.S), f)[0])

#  文件重命名

In [10]:
#     os.remove(os.path.join(src_path,i))   os.rename(srcFile,dstFile)
# import os,shutil
#     shutil.move     shutil.copy(source, target)
path = '/home/zut_csi/tomding/RCNN/data_gait/all'
for i in os.listdir(path):
    if not i.startswith('room'):
        os.rename(os.path.join(path, i), os.path.join(path, 'room1-'+i))
#     if i.startswith('room'):
#         os.rename(os.path.join(path, i), os.path.join(path, i.replace('room1-', '')))

# 根据天线整理数据

In [8]:
path = r'/home/zut_csi/tomding/RCNN/data_gait/all/'
for f in os.listdir(path):
    if f.endswith('.dat') or f.endswith('.txt'):
        #print(f)
        rx = re.findall(re.compile('user.*-.*-.*-r(.*?)\..*', re.S), f)[0]
        #print(rx)
        shutil.move(os.path.join(path, f), 
                    os.path.join(path, str(rx), f))
# X_train = process_data(train_raw_file_path, train_save_path)
# test_raw_file_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/test'

In [3]:
# path = r'/home/zut_csi/tomding/RCNN/data_gait/room1'
# for f in os.listdir(path):
#     if f.endswith('.dat'):
#         #print(f)
#         rx = re.findall(re.compile('user.*-.*-.*-r(.*?)\.dat', re.S), f)[0]
#         #print(rx)
#         shutil.move(os.path.join(path, f), 
#                     os.path.join(path, str(rx), f))
# # X_train = process_data(train_raw_file_path, train_save_path)
# # test_raw_file_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/test'

In [4]:
3317+3315+3318+3313+3313+3313

19889

# 随机划分训练集测试集

In [5]:
# id-a-b-c-d-rx.dat
# 'id' represents user's id,    2
# 'a' represents room id,        1-10
# 'b' represents position id,     1-5
# 'c' represents orientation id,   1-5
# 'd' represents instance id,       1-10
# 'rx' range between 'r1' and 'r6', corresponding to 6 receivers.   1-6

# import os 
# import shutil
# import random 
# file_list = os.listdir('/home/zut_csi/tomding/RCNN/data_qinghua/20181112/user2')
# for f in file_list:
#     if f.endswith('.dat'): # 文件
#         # 随机选取部分作为测试
#         rand = random.randint(1, 10)
#         filename1 = os.path.splitext(f)[1]  # 读取文件后缀名
#         filename0 = os.path.splitext(f)[0]  # 读取文件名
#         m = filename1 == '.jpg'
#         print(m)
#         if m :
#             full_path = os.path.join(path1, files)
#             despath = path2 + filename0+'.jpg' #.jpg为你的文件类型，即后缀名，读者自行修改
#             shutil.move(full_path, despath)

#         else :
#             continue

# 提取CSI中想要的部分

In [12]:
# %%time
# # 提取CSI中想要的部分
# def processor(raw_file_path, f, save_path):
#     file_name = os.path.join(raw_file_path, f)
#     extracted_data = extract_csi(file_name)
    
#     # print('processing {} the length of this file is:{}'.format(file_name, len(extracted_data)))
#     try:
#         tx, rx, sub = extracted_data[0]['csi'].shape
#     except:
#         print(file_name, len(extracted_data))
#     data_csi = np.zeros((len(extracted_data), tx, rx, sub), dtype=np.complex64)
#     # qinghua (1938, 1, 3, 30)  ;    self
#     for i in range(len(extracted_data)):
#         data_csi[i] = get_scaled_csi(extracted_data[i])
#     data_csi = np.clip(np.abs(np.squeeze(data_csi)), 1e-8, 1e100).reshape(-1, tx*rx, 30)   
#     data = np.zeros((data_csi.shape[0], tx*rx))  # N*4
#     for ant in range(tx*rx):  # 每个天线对上的CSI变化趋势相同,为节约计算这里选择天线对即可
#         data_csi_ant = data_csi[:, ant, :]
#         b, a = signal.butter(5, 4*2/30, 'low')
#         var_max = 0
#         s_max = None
#         for s in range(30):
#             carrier_data = signal.lfilter(b, a, data_csi_ant[:, s])  # N*1
#             length = len(carrier_data)
#             var_temp = np.var(carrier_data[length//5:3*length//5])
#             if var_max < var_temp:
#                 var_max = var_temp
#                 s_max = carrier_data
#         data[:, ant] = s_max
#     scio.savemat(os.path.join(save_path, f.replace('dat', 'mat')), {'csi': data})


# raw_file_path, save_path = '/home/zut_csi/tomding/RCNN/data_gait/all', '/home/zut_csi/tomding/RCNN/data_processed/all'
# #'/home/zut_csi/tomding/RCNN/da', '/home/zut_csi/tomding/RCNN/d'
# X = {}
# thread_list= []
# for f in os.listdir(raw_file_path):             
#     if f.endswith('.dat'):
#         # X[f] = data
#         #processor(raw_file_path, f, save_path,)
#         t = threading.Thread(target=processor, args=(raw_file_path, f, save_path,))                              
#         t.setDaemon(True)  # 设置为守护线程                              
#         thread_list.append(t)
#     else:
#         pass
# for t in tqdm(thread_list):         
#     t.start()  # 启动线程     
# for t in tqdm(thread_list): 
#     t.join()  # 等待子线程结束
# print('-----------------------------------all raw file processed---------------------------------')
#     # return X

100%|██████████| 22017/22017 [00:27<00:00, 801.42it/s] 

-----------------------------------all raw file processed---------------------------------
CPU times: user 3h 22min 30s, sys: 11min 10s, total: 3h 33min 41s
Wall time: 3h 11min 6s


# 生成3D_Conv数据

In [16]:
%%time
# 提取CSI中想要的部分
def processor(raw_file_path, f, save_path, max_file_len):
    file_name = os.path.join(raw_file_path, f)
    extracted_data = extract_csi(file_name)
    
    # print('processing {} the length of this file is:{}'.format(file_name, len(extracted_data)))
    try:
        tx, rx, sub = extracted_data[0]['csi'].shape
    except:
        print(file_name, len(extracted_data))
    data_csi = np.zeros((max_file_len, tx, rx, sub), dtype=np.complex64)
    # qinghua (1938, 1, 3, 30)  ;    self
    for i in range(len(extracted_data)):
        data_csi[i] = get_scaled_csi(extracted_data[i])
    data_csi = np.clip(np.abs(data_csi), 1e-8, 1e100).transpose(1, 2, 3, 0)     
    for r in range(rx):  # 每个天线对上的CSI变化趋势相同,为节约计算这里选择天线对即可
        b, a = signal.butter(5, 4*2/30, 'low')
        var_max = 0
        for s in range(30):
            data_csi[0, r, s, :] = signal.lfilter(b, a, data_csi[0, r, s, :])  # N*1
    scio.savemat(os.path.join(save_path, f.replace('dat', 'mat')), {'csi': data_csi})


raw_file_path, save_path = '/home/zut_csi/tomding/RCNN/data_gait/all/1', '/home/zut_csi/tomding/RCNN/data_processed/test'
#'/home/zut_csi/tomding/RCNN/da', '/home/zut_csi/tomding/RCNN/d'
X = {}
thread_list= []
max_file_len = 3641
         
for f in os.listdir(raw_file_path):             
    if f.endswith('r1.dat'):
        # X[f] = data
        #processor(raw_file_path, f, save_path,)
        t = threading.Thread(target=processor, args=(raw_file_path, f, save_path, max_file_len, ))                              
        t.setDaemon(True)  # 设置为守护线程                              
        thread_list.append(t)
    else:
        pass
for t in tqdm(thread_list):         
    t.start()  # 启动线程     
for t in tqdm(thread_list): 
    t.join()  # 等待子线程结束
print('-----------------------------------all raw file processed---------------------------------')
    # return X

 16%|█▌        | 580/3672 [06:01<32:07,  1.60it/s]  


KeyboardInterrupt: 

In [17]:
%%time
raw_file_path, save_path = '/home/zut_csi/tomding/RCNN/data_gait/all/1', '/home/zut_csi/tomding/RCNN/data_processed/test'
#'/home/zut_csi/tomding/RCNN/da', '/home/zut_csi/tomding/RCNN/d'
X = {}
thread_list= []
max_file_len = 3641
import multiprocessing
# 多进程任务
pool = multiprocessing.Pool(processes=4)
result= []
for f in tqdm(os.listdir(raw_file_path)):             
    if f.endswith('r1.dat'):
        result.append(pool.apply_async(processor, (raw_file_path, f, save_path, max_file_len, )))
pool.close()
pool.join()
ans = [res.get() for res in result]
print(ans)

100%|██████████| 3672/3672 [00:00<00:00, 14589.52it/s]


1
1
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None,